In [4]:
import os
import sys
import numpy as np
os.chdir("..")
print(os.getcwd())
sys.path.insert(0, os.getcwd())

import tqdm
import torch
import trimesh
import skimage
import skimage.measure
import csv
from utils import load_imf
from trainers.utils.diff_ops import gradient

/home/weidemaier/New Network


In [18]:
path = "/home/weidemaier/New Network/NFGP/logs/create_neural_fields_data.path=-home-weidemaier-NFGP-sphere2.npy_2024-Sep-21-16-59-11"

net, cfg = load_imf(path, return_cfg=True)

/home/weidemaier/New Network/NFGP/logs/create_neural_fields_data.path=-home-weidemaier-NFGP-sphere2.npy_2024-Sep-21-16-59-11/checkpoints/epoch_99_iters_99400.pt
Net:
Net(
  (blocks): ModuleList(
    (0): Linear(in_features=3, out_features=128, bias=True)
    (1): Linear(in_features=128, out_features=128, bias=True)
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): Linear(in_features=128, out_features=128, bias=True)
    (4): Linear(in_features=128, out_features=128, bias=True)
    (5): Linear(in_features=128, out_features=1, bias=True)
  )
  (act): Sine()
)
Scheduler Type: adaptive


In [6]:
#Plot slice of learned function (both for heat and 2nd step)

sample_size = 100000

linsp = np.linspace(-1.5, 1.5, 200)
vec = [None]*200**2

i = 0
while i < 200:
    k = 0
    while k < 200:
        ### if different slices should be considered, change here
        vec[i*200 + k] = [0, linsp[i], linsp[k]]    
        k = k+1
    i = i+1

xyz = torch.tensor(np.float32(vec)).cuda().requires_grad_(True)
vec = net(xyz)
grad = gradient(vec, xyz)
norm_grad = grad.norm(p = 2, dim=-1).view(200**2, 1)
normed = (grad/norm_grad).view(200**2, 3)

a = vec.detach().cpu().numpy()
A = xyz.detach().cpu().numpy()

i = 0
vec = [None]*200**2
while i < 200**2:
    vec[i] = [A[i, 1], A[i,2], a[i,0]]
    i = i+1
np.savetxt("plot_3d_new.csv", vec , delimiter = ",", header = "x,y,z")

In [21]:
#Code for error estimation of learned SDF using input mesh
from trimesh import proximity

def err_mesh(net, mesh, bs):
    normals = mesh.vertex_normals
    
    sample = trimesh.sample.sample_surface(mesh, bs)
    points = sample[0]
    coords = points
    coords -= np.mean(coords, axis=0, keepdims=True)
    keep_aspect_ratio = True
    if keep_aspect_ratio:
        coord_max = np.amax(coords)
        coord_min = np.amin(coords)
    else:
        coord_max = np.amax(coords, axis=0, keepdims=True)
        coord_min = np.amin(coords, axis=0, keepdims=True)
    print((coord_max - coord_min))
    coords = (coords - coord_min) / (coord_max - coord_min)
    coords -= 0.5
    coords *= 2.
    points = coords
    face_count = sample[1]
    face_normals = mesh.face_normals
    normals = face_normals[face_count]
    mesh_prox = trimesh.proximity.ProximityQuery(mesh)
    idx = np.random.randint(points.size/3, size=bs)
    #B = points[idx,:]
    #A = normals[idx,:]
    #normals = A
    points = np.float32(points)
    points = torch.tensor(points)
    points = points.cuda()
    points.requires_grad = True
    normals = np.float32(normals)
    normals = torch.tensor(normals)
    normals = normals.cuda()
    normals.requires_grad = True
    val = net(points)
    

    on_loss = abs(val).mean()
    on_max = torch.max(abs(val))
    #calc dists to mesh
    r = np.random.uniform(0.15, 0.5, bs)
    
    phi = np.random.uniform(0, np.pi, bs)
    theta = np.random.uniform(0, 2*np.pi, bs)
    i = 0
    random_points = [None]*bs
    while (i < bs):
        x = r[i]*np.sin(theta[i])*np.cos(phi[i])
        y = r[i]*np.sin(theta[i])*np.sin(phi[i])
        z = r[i]*np.cos(theta[i])
        random_points[i] = [x,y,z]
        i = i+1
    
    dist = proximity.closest_point(mesh, random_points)
    dist = random_points - dist[0]
    dist = np.linalg.norm(dist,axis=1)
    
    dist = np.float32(dist)
    dist = torch.tensor(dist)
    dist = dist.cuda()
    dist.requires_grad = True
    random_points = np.float32(random_points)
    random_points = torch.tensor(random_points)
    random_points = random_points.cuda()
    random_points.requires_grad = True
    val_off = net(random_points)
    
    
    off_loss = abs(val_off - dist).mean()
    off_max = torch.max(abs(val_off - dist))
    grad = gradient(val, points).view(bs, 3)
    l_normalized = grad/grad.norm(dim = -1).view(bs, 1)
    sc_prod = l_normalized* (normals)
    sc_prod = torch.sum(sc_prod.squeeze(), dim = 1)
    sc_prod = torch.ones_like(sc_prod) + sc_prod
    sc_max = torch.max(sc_prod)
    sc_prod = torch.mean(sc_prod)
    return on_loss.item(), on_max.item(), off_loss.item(), off_max.item(), sc_prod.item(), sc_max.item()


In [22]:
mesh = trimesh.load("/home/weidemaier/siren-master/rm_mid.obj")
print(err_mesh(net, mesh, 50))

0.824763627984178
(0.08867190033197403, 0.24954932928085327, 0.2713235318660736, 0.7658616900444031, 0.1041562408208847, 1.1373871564865112)
